In [ ]:
#Logging in to db
import pyodbc
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import sklearn.utils
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

#connecting to DB
    #>>>cellsConn = pyodbc.connect()

#Load data


#Getting Cell loads
 #>>>cellsConn = pyodbc.connect()


did=data['did'].values
def get_Load_stats(did):        
        query1 = '({})'.format(''.join(["x = '{}' OR ".format(a) for a in did])[:-4])
        query = (#>>> SQL Query
            .format(query1)
                 )
        Load_stats =(pd.read_sql(query, cellsConn))
        return Load_stats
load=get_Load_stats(did)
load.drop(['date'],axis=1,inplace=True)
data2=data.merge(load, on='did', how='left')

#Getting GIDs
did=data2['did']
def get_Gids(did):        
        query1 = '({})'.format(''.join(["x = '{}' OR ".format(a) for a in did])[:-4])
        query = (#>>> SQL Query.format(query1)
                 )
        Gids =(pd.read_sql(query, cellsConn))
        return Gids
Gids=get_Gids(did)
df=data2.merge(Gids, on='did', how='inner')
df['Site_Name']=df['Site_Name'].fillna(df['Cell_name'].str[:-2])

#Clustering
df.dropna(inplace=True)
df['Load status']=df['Overload Flag'].apply(lambda x: 'heavy load' if x>=1 else 'low load')
Lamp=df[df['Cell_name'].str.contains('LAMP')]
IBS=df[df['Cell_name'].str.contains('IBS')]
sites=df[~df['Cell_name'].str.contains('LAMP') & ~df['Cell_name'].str.contains('IBS')]
load=sites[sites['Overload Flag']==1]

sites=sites[sites['Overload Flag']!=1]

U900C1=sites[sites['BAND']=='U900-C1']
U900C2=sites[sites['BAND']=='U900-C2']
U2100C1=sites[sites['BAND']=='U2100-C1']
U2100C2=sites[sites['BAND']=='U2100-C2']

sklearn.utils.check_random_state(1000)

def get_clusters(X):
    Clus_dataSet =X[['LONGITUDE','LATITUDE']]
    Clus_dataSet = np.nan_to_num(Clus_dataSet)
    """The sites must be within 2KM(this could vary from network to network depending on the inter-site distance) 
    of each other and a cluster must contain at least 5 points"""
    # Compute DBSCAN
    epsilon =2/6371.0088
    db = DBSCAN(eps=epsilon, min_samples=5, algorithm='ball_tree', metric='haversine').fit(np.radians(Clus_dataSet))
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    X["Clus_Db"]=labels

    return(X)

get_clusters(U900C1)
get_clusters(U900C2)

get_clusters(U2100C1)
get_clusters(U2100C2)

U900C1_clusters=get_clusters(U900C1)
U900C2_clusters=get_clusters(U900C2)
U2100C1_clusters=get_clusters(U2100C1)
U2100C2_clusters=get_clusters(U2100C2)

final=U900C1_clusters.append([U900C2_clusters,U2100C1_clusters,U2100C2_clusters])

external1=final[final['Clus_Db']>-1]

internal1=final[final['Clus_Db']==-1]

# internal1['New_column'].head()
internal1['New_column']=internal1['Cell_name'].str[-2:].replace({'-4':'-0', '-5':'-1','-6':'-2'})
internal1.loc[:,'New_column']=internal1['Site_Name']+ internal1['New_column']

ids = internal1["New_column"]
maintenance=internal1[ids.isin(ids[ids.duplicated()])]
maintenance=maintenance.sort_values('Cell_name', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

def difference(df1, df2, which=None):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(df2,
                              indicator=True,
                              how='outer')
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    return diff_df

diff_df=difference(internal1,maintenance)
Lamp.rename(columns={"Load status": "Status"}, inplace=True)
IBS.rename(columns={"Load status": "Status"}, inplace=True)
load.rename(columns={"Load status": "Status"}, inplace=True)
load['Status']='Overloaded'
diff_df['Status']='External'

# diff_df.head()
maintenance['Status']='Maintenance'

external1['Status']='External'

#Final dataset
Output=diff_df.append([load,maintenance,external1,Lamp,IBS])

# Writting to DB
conn = pyodbc.connect( #>>> Connect to DB
  )
cursor = conn.cursor()

cursor=conn.cursor()
for index, row in Output.iterrows():
    cursor.execute(#>>> Insert Data to the DB           )
    conn.commit()
cursor.close()
conn.close()